# Load and compare predictions

In [1]:
import numpy as np

import pandas as pd

import tensorflow_datasets as tfds

c:\Users\angso\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('model1.npy', 'rb') as f:
    model1 = np.load(f)

with open('model2.npy', 'rb') as f:
    model2 = np.load(f)

with open('model3.npy', 'rb') as f:
    model3 = np.load(f)

with open('model4.npy', 'rb') as f:
    model4 = np.load(f)

In [3]:
val_ds = tfds.load('squad/v1.1')['validation']

def extract_data(instance):
    paragraph = instance['context']
    question = instance['question']
    answer = instance['answers']['text'][0]
    answer_start = instance['answers']['answer_start'][0]
    return paragraph, question, answer, answer_start

val_ds = pd.DataFrame(val_ds.map(extract_data).as_numpy_iterator())
headers = ['Paragraph', 'Question', 'Answer', 'Answer Start']
val_ds.columns = headers

In [4]:
new_ds = []

for i, row in val_ds.iterrows():
    paragraph, question, answer, answer_start = row['Paragraph'], row['Question'], row['Answer'], row['Answer Start']
    paragraph = paragraph.decode('utf-8')
    question = question.decode('utf-8')
    answer = answer.decode('utf-8')
    
    answer_end = answer_start + len(answer)
    if paragraph[answer_start-1:answer_end-1] == answer:
        answer_start -= 1
        answer_end -= 1
    elif paragraph[answer_start-2:answer_end-2] == answer:
        answer_start -= 2
        answer_end -= 2

    m1_start = model1[i][0]
    m1_end = model1[i][1]
    m1 = paragraph[m1_start:m1_end+1]
    m2_start = model2[i][0]
    m2_end = model2[i][1]
    m2 = paragraph[m2_start:m2_end+1]
    m3_start = model3[i][0]
    m3_end = model3[i][1]
    m3 = paragraph[m3_start:m3_end+1]
    m4_start = model4[i][0]
    m4_end = model4[i][1]
    m4 = paragraph[m4_start:m4_end+1]

    new_ds.append([paragraph, question, answer, m1, m2, m3, m4, answer_start, answer_end, m1_start, m1_end, m2_start, m2_end, m3_start, m3_end, m4_start, m4_end])

In [5]:
val_ds = pd.DataFrame(new_ds)
headers = ['Paragraph', 'Question', 'Answer', 'M1', 'M2', 'M3', 'M4', 'Answer Start', 'Answer End', 'M1 Start', 'M1 End', 'M2 Start', 'M2 End', 'M3 Start', 'M3 End', 'M4 Start', 'M4 End']
val_ds.columns = headers
val_ds.head()

,Paragraph,Question,Answer,M1,M2,M3,M4,Answer Start,Answer End,M1 Start,M1 End,M2 Start,M2 End,M3 Start,M3 End,M4 Start,M4 End
0,The Calvin cycle starts by using the enzyme Ru...,How many G3P molecules leave the cycle?,one out of every six,o,o,o,one out of every six,479,499,85,85,85,85,85,85,479,499
1,A cylindrical Service Module (SM) supported th...,What type of antenna was used for communicatio...,high-gain S-band antenna,cylindrical Service Module (SM),rte,cylindrical Service Module (SM) support,high-gain S-band,218,242,1,32,39,41,1,40,218,234
2,"The addition of new rock units, both depositio...",Where do dikes form?,in areas that are being actively deformed,"ositionally and intrusively, often occurs duri...",po,r deformational processes result in the creati...,"along cracks,",826,867,40,153,173,174,119,177,770,782
3,The mouth of the Rhine into Lake Constance for...,The inland delta at the mouth of the Rhine is ...,Lake Constance,forms an inland delta. The delta is delimite,ut,uth of,Lake Constance,28,42,43,86,6,7,6,11,28,42
4,A computational problem can be viewed as an in...,Is a problem instance typically characterized ...,concrete,computational problem can be viewed as an inf...,c,so,concrete,402,410,1,73,73,73,93,94,402,410


In [6]:
val_ds.to_csv('predictions.csv', index=False)